In [1]:
import numpy as np
import pandas as pd
# Load the datasets
df = pd.read_csv("https://raw.githubusercontent.com/wongwara/Jobseeker_Baymax/main/dataset/listings2019_2022_salary_adjust01052023.csv")

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3902 entries, 0 to 3901
Data columns (total 52 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   jobId                  3902 non-null   int64  
 1   jobTitle               3902 non-null   object 
 2   jobClassification      3902 non-null   object 
 3   jobSubClassification   3902 non-null   object 
 4   advertiserName         3902 non-null   object 
 5   advertiserId           3902 non-null   int64  
 6   companyId              1067 non-null   float64
 7   companyName            1476 non-null   object 
 8   companyRating          1067 non-null   float64
 9   listingDate            3902 non-null   object 
 10  expiryDate             3902 non-null   object 
 11  teaser                 3374 non-null   object 
 12  nation                 3902 non-null   object 
 13  state                  3902 non-null   object 
 14  city                   3902 non-null   object 
 15  area

In [4]:
df = df[['jobClassification','teaser','nation','workType','salary_string','isRightToWorkRequired','desktopAdTemplate',
         'Python','SQL','R','Tableau','SAS','Matlab','Hadoop','Spark','Java', 'Scala','recruiter']]

In [5]:
len(df['jobClassification'].unique())

25

In [6]:
df = df[df['nation'].str.contains('Australia')]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3897 entries, 0 to 3901
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   jobClassification      3897 non-null   object
 1   teaser                 3369 non-null   object
 2   nation                 3897 non-null   object
 3   workType               3897 non-null   object
 4   salary_string          1512 non-null   object
 5   isRightToWorkRequired  3411 non-null   object
 6   desktopAdTemplate      3203 non-null   object
 7   Python                 3897 non-null   int64 
 8   SQL                    3897 non-null   int64 
 9   R                      3897 non-null   int64 
 10  Tableau                3897 non-null   int64 
 11  SAS                    3897 non-null   int64 
 12  Matlab                 3897 non-null   int64 
 13  Hadoop                 3897 non-null   int64 
 14  Spark                  3897 non-null   int64 
 15  Java                 

In [8]:
df = df.drop(['nation'],axis =1)

In [9]:
df.isna().sum() 

jobClassification           0
teaser                    528
workType                    0
salary_string            2385
isRightToWorkRequired     486
desktopAdTemplate         694
Python                      0
SQL                         0
R                           0
Tableau                     0
SAS                         0
Matlab                      0
Hadoop                      0
Spark                       0
Java                        0
Scala                       0
recruiter                   0
dtype: int64

In [10]:
df["salary_string"].describe

<bound method NDFrame.describe of 0                                               NaN
1                                             Super
2                        $90000 - $120000 per annum
3                             $90000 - $110000 p.a.
4                                               NaN
                           ...                     
3897    Open to Quote (Sydney or Canberra Location)
3898                                            NaN
3899                                Desirable Rates
3900                                            NaN
3901                                            NaN
Name: salary_string, Length: 3897, dtype: object>

In [11]:
df.head()

,jobClassification,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,SAS,Matlab,Hadoop,Spark,Java,Scala,recruiter
0,Science & Technology,My client is a leading Australian-owned and ra...,Full Time,NaN,f,\n \n \n Senior Method Development Im...,0,0,1,0,0,0,0,0,0,0,1
1,Information & Communication Technology,The Data Scientist will be responsible for mon...,Full Time,Super,t,\n \n \n \n Pricing Data Scientist...,0,0,1,0,1,0,0,0,0,0,1
2,Information & Communication Technology,Fantastic organisation seeks experienced Insig...,Full Time,$90000 - $120000 per annum,f,\n \n \n \n Insights Analyst – Onl...,0,1,1,0,0,0,0,0,0,0,1
3,Banking & Financial Services,This role requires an individual with strong c...,Full Time,$90000 - $110000 p.a.,f,\n \n \n \n Credit Risk Analyst \n...,0,1,1,0,1,0,0,0,0,0,1
4,Information & Communication Technology,"Our client, a large Sydney based telecommunica...",Contract/Temp,NaN,f,\n \n \n \n Data Scientist \n ...,1,1,0,1,0,0,0,0,0,0,0


<!-- https://www.kaggle.com/code/nomilk/deep-exploration-of-data-science-job-listings/script -->

Looks like a lot of jobs don't reveal the salary (i.e. `""` empty string values). There's a lot of interesting data in the non-missing values, but the values are all in character strings! To enable us to do something interesting with this data, we will first need to convert it to numeric data. 

In Australia, [full time workers are paid at least $740.80 for a 38-hour week](https://www.business.gov.au/People/Pay-and-conditions/Employees-pay-leave-and-entitlements), so the `exclude_below` parameter is set to `740.80 * 48`.


In [12]:
import re

def extract_salary(text):    
    range_pattern = r'([\d\.]+) *- *\$?([\d\.]+)'
    range_matches = re.search(range_pattern, text.replace(",", ""))
    if range_matches:
        lo, hi = range_matches.groups()
        salary_range = (float(lo), float(hi))
    else:
        salary_range = None
    
    return salary_range


In the above code, the find_range function is defined to extract the lower and upper salary range from a string that starts with a dollar sign. The function first checks if the string starts with a dollar sign, and if so, uses a regular expression to extract the lower and upper salary range from the string. The function returns the lower and upper salary range as a tuple. If the string does not start with a dollar sign, the function returns None, None.

The find_range function is then applied to the "salary_string" column of the pandas DataFrame using the .apply() method. The resulting tuple is then expanded into two separate columns "lower_salary" and "upper_salary" using the .apply(pd.Series) method.

Note that the regular expression used in the find_range function may need to be adjusted depending on the specific format of the salary strings in your DataFrame.

In [13]:
df = df[df['salary_string'].notna()]

In [14]:
df["salary_string"] = df["salary_string"].apply(extract_salary)

In [15]:
df.head(5)

,jobClassification,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,SAS,Matlab,Hadoop,Spark,Java,Scala,recruiter
1,Information & Communication Technology,The Data Scientist will be responsible for mon...,Full Time,None,t,\n \n \n \n Pricing Data Scientist...,0,0,1,0,1,0,0,0,0,0,1
2,Information & Communication Technology,Fantastic organisation seeks experienced Insig...,Full Time,"(90000.0, 120000.0)",f,\n \n \n \n Insights Analyst – Onl...,0,1,1,0,0,0,0,0,0,0,1
3,Banking & Financial Services,This role requires an individual with strong c...,Full Time,"(90000.0, 110000.0)",f,\n \n \n \n Credit Risk Analyst \n...,0,1,1,0,1,0,0,0,0,0,1
7,Banking & Financial Services,One of Australia's leading financial service p...,Full Time,"(110000.0, 120000.0)",t,\n \n \n Data Analytics Recruitment Sol...,1,1,1,0,1,0,0,0,0,0,1
8,Information & Communication Technology,Awesome opportunity for a highly skilled and e...,Full Time,None,t,\n \n \n Senior Data Scientist \n \n...,1,0,0,0,0,0,1,0,0,0,1


In [16]:
df = df[df['salary_string'] != (None,None)]

In [17]:
df.head()

,jobClassification,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,SAS,Matlab,Hadoop,Spark,Java,Scala,recruiter
1,Information & Communication Technology,The Data Scientist will be responsible for mon...,Full Time,None,t,\n \n \n \n Pricing Data Scientist...,0,0,1,0,1,0,0,0,0,0,1
2,Information & Communication Technology,Fantastic organisation seeks experienced Insig...,Full Time,"(90000.0, 120000.0)",f,\n \n \n \n Insights Analyst – Onl...,0,1,1,0,0,0,0,0,0,0,1
3,Banking & Financial Services,This role requires an individual with strong c...,Full Time,"(90000.0, 110000.0)",f,\n \n \n \n Credit Risk Analyst \n...,0,1,1,0,1,0,0,0,0,0,1
7,Banking & Financial Services,One of Australia's leading financial service p...,Full Time,"(110000.0, 120000.0)",t,\n \n \n Data Analytics Recruitment Sol...,1,1,1,0,1,0,0,0,0,0,1
8,Information & Communication Technology,Awesome opportunity for a highly skilled and e...,Full Time,None,t,\n \n \n Senior Data Scientist \n \n...,1,0,0,0,0,0,1,0,0,0,1


In [18]:
df = df[df['salary_string'].notna()]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817 entries, 2 to 3893
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   jobClassification      817 non-null    object
 1   teaser                 695 non-null    object
 2   workType               817 non-null    object
 3   salary_string          817 non-null    object
 4   isRightToWorkRequired  704 non-null    object
 5   desktopAdTemplate      695 non-null    object
 6   Python                 817 non-null    int64 
 7   SQL                    817 non-null    int64 
 8   R                      817 non-null    int64 
 9   Tableau                817 non-null    int64 
 10  SAS                    817 non-null    int64 
 11  Matlab                 817 non-null    int64 
 12  Hadoop                 817 non-null    int64 
 13  Spark                  817 non-null    int64 
 14  Java                   817 non-null    int64 
 15  Scala                 

In [20]:
# df.to_csv('df_salary_cleaned.csv')

In [21]:
df['isRightToWorkRequired'].unique()

array(['f', 't', nan], dtype=object)

In [22]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode

# Define a function to get mode of the column
def get_mode(column):
    return mode(column, nan_policy='omit')[0][0]

# Replace the NaN values with the mode of the column
mode_val = get_mode(df['isRightToWorkRequired'])
df['isRightToWorkRequired'] = df['isRightToWorkRequired'].fillna(mode_val)

# Create the LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder object to the column
le.fit(df['isRightToWorkRequired'])

# Transform the column to label encoding
column = le.transform(df['isRightToWorkRequired'])


In [23]:
# Import OrdinalEncoder from sklearn.preprocessing
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OneHotEncoder

In [24]:
df['isRightToWorkRequired'].unique()

array(['f', 't'], dtype=object)

In [25]:
df['isRightToWorkRequired'] = df['isRightToWorkRequired'].replace({'f': 0, 't': 1})

In [26]:
df['workType'].unique()

array(['Full Time', 'Contract/Temp', 'Part Time', 'Casual/Vacation',
       'Full time', 'Part time'], dtype=object)

In [27]:
df['workType'] = df['workType'].replace({'Full time': 'Full Time', 'Part time': 'Part Time'})

In [28]:
df['workType'].unique()

array(['Full Time', 'Contract/Temp', 'Part Time', 'Casual/Vacation'],
      dtype=object)

In [29]:
# gender is a dummy variables ( 0 = Male and 1 = Female)
workType_cats = [['Full Time', 'Contract/Temp', 'Part Time', 'Casual/Vacation']]
Label = LabelEncoder()
# Use our trained encoder to transform this column
df['workType_encoded'] = Label.fit_transform(df[['workType']])

# display the columns
df[['workType','workType_encoded']]

df['workType']= df['workType_encoded']

In [30]:
jobClassification_cats = [['Information & Communication Technology',
       'Banking & Financial Services', 'Science & Technology',
       'Education & Training', 'Government & Defence',
       'Consulting & Strategy', 'Healthcare & Medical',
       'Human Resources & Recruitment', 'Marketing & Communications',
       'Retail & Consumer Products', 'Administration & Office Support',
       'Accounting', 'Insurance & Superannuation',
       'Mining, Resources & Energy', 'Real Estate & Property',
       'Manufacturing, Transport & Logistics', 'Engineering']]
jobClassification_cats_enc =OrdinalEncoder(categories=jobClassification_cats)
# Use our trained encoder to transform this column
df['jobClassification_encoded'] = jobClassification_cats_enc.fit_transform(df[['jobClassification']])

# display the columns
df[['jobClassification','jobClassification_encoded']]

df['jobClassification'] = df['jobClassification_encoded']

In [31]:
df['teaser']= df['teaser'].fillna('')


In [32]:
df['desktopAdTemplate']= df['desktopAdTemplate'].fillna('')

In [33]:
import re 
# Define pipeline function to perform multiple cleaning steps on a given text
def clean_text(text):
    text = re.sub(r'[^\w]', ' ', str(text)) # Remove symbols
    text = re.sub(r'[ ]{2,}', ' ', str(text)) # Remove extra spaces
    text = re.sub(r'[ \t]+$', '', str(text)) # Remove trailing white spaces
    text = re.sub(r'\s+', ' ', text) # Remove newline (\n)
    text = re.sub(r"\'", "", text) # Remove quotes
    return text

In [34]:
# "jobTitle", "teaser", "mobileAdTemplate" and "desktopAdTemplate"
df["teaser_cleaned"] = df["teaser"].apply(clean_text)
df["desktopAdTemplate_cleaned"] = df["desktopAdTemplate"].apply(clean_text)

In [35]:
df["teaser_cleaned"] = df["teaser_cleaned"].str.lower()
df["desktopAdTemplate_cleaned"] = df["desktopAdTemplate_cleaned"].str.lower()

In [36]:
df["teaser_cleaned"] = df["teaser"].to_string()
df["desktopAdTemplate_cleaned"] = df["desktopAdTemplate"].to_string()

In [37]:
# Function to remove stopwords
import string 
from nltk.corpus import stopwords
# Function to remove stopwords
stop_words = stopwords.words('english')

def remove_stopwords(text):
    useful_words = []
    for i in text.split():
        if i.strip().lower() not in stop_words:
            useful_words.append(i.strip())
    return " ".join(useful_words)

In [38]:
df["teaser_cleaned"] = df["teaser_cleaned"].apply(remove_stopwords)
df["desktopAdTemplate_cleaned"] = df["desktopAdTemplate_cleaned"].apply(remove_stopwords)

In [39]:
df['teaser_word_count'] = df['teaser_cleaned'].apply(lambda x: len(str(x).split(" ")))
df['desktopAdTemplate_word_count'] = df['desktopAdTemplate_cleaned'].apply(lambda x: len(str(x).split(" ")))
df[['teaser_word_count', 'desktopAdTemplate_word_count']].sample(5)

,teaser_word_count,desktopAdTemplate_word_count
1850,4638,5209
51,4638,5209
647,4638,5209
3210,4638,5209
2917,4638,5209


In [40]:
df[['teaser_word_count','desktopAdTemplate_word_count']].describe()

,teaser_word_count,desktopAdTemplate_word_count
count,817.0,817.0
mean,4638.0,5209.0
std,0.0,0.0
min,4638.0,5209.0
25%,4638.0,5209.0
50%,4638.0,5209.0
75%,4638.0,5209.0
max,4638.0,5209.0


In [41]:
from nltk.tokenize import word_tokenize #Tokenizing a sentence (breaking it into individual “words”) 
from nltk.stem import PorterStemmer

# Convert to string
text_teaser = df["teaser_cleaned"].to_string()
text_desktop = df["desktopAdTemplate_cleaned"].to_string()

# Using the Porter Stemmer
# allows us to remove the prefixes, suffixes from a word and and change it to its base form
porter = PorterStemmer()
text_teaser = porter.stem(text_teaser)
text_desktop = porter.stem(text_desktop)

In [42]:
text_teaser = word_tokenize(text_teaser)
text_desktop = word_tokenize(text_desktop)

In [43]:
df.sample(5)

,jobClassification,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,...,Spark,Java,Scala,recruiter,workType_encoded,jobClassification_encoded,teaser_cleaned,desktopAdTemplate_cleaned,teaser_word_count,desktopAdTemplate_word_count
180,4.0,Research new machine learning methods such as ...,1,"(134421.0, 148725.0)",1,\n \n \n \n Data Scientist \n ...,1,0,1,0,...,0,1,1,0,1,4.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
2515,2.0,These roles are permanent positions in our Met...,2,"(101751.0, 117622.0)",0,The Australian Bureau of Statistics (ABS) is l...,1,1,0,0,...,0,1,0,0,2,2.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
3370,0.0,The Australian Signals Directorate is seeking ...,2,"(69811.0, 95873.0)",0,Do you wish you could use your cultural and la...,0,1,0,1,...,0,0,0,0,2,0.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
3584,2.0,,2,"(110000.0, 149999.0)",0,About IMDEXIMDEX is a leading global Mining-Te...,1,1,1,1,...,0,0,0,0,2,2.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
3411,0.0,,2,"(140000.0, 150000.0)",0,The Client:Our client is a well-established or...,1,1,1,1,...,0,1,0,1,2,0.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209


In [44]:
df.head()

,jobClassification,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,...,Spark,Java,Scala,recruiter,workType_encoded,jobClassification_encoded,teaser_cleaned,desktopAdTemplate_cleaned,teaser_word_count,desktopAdTemplate_word_count
2,0.0,Fantastic organisation seeks experienced Insig...,2,"(90000.0, 120000.0)",0,\n \n \n \n Insights Analyst – Onl...,0,1,1,0,...,0,0,0,1,2,0.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
3,1.0,This role requires an individual with strong c...,2,"(90000.0, 110000.0)",0,\n \n \n \n Credit Risk Analyst \n...,0,1,1,0,...,0,0,0,1,2,1.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
7,1.0,One of Australia's leading financial service p...,2,"(110000.0, 120000.0)",1,\n \n \n Data Analytics Recruitment Sol...,1,1,1,0,...,0,0,0,1,2,1.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
10,2.0,Postdoctoral researcher in molecular evolution...,2,"(71509.0, 90215.0)",0,\n \n \n \n Postdoctoral Fellow \n...,0,0,0,0,...,0,1,0,0,2,2.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
11,3.0,Postdoctoral researcher in molecular evolution...,2,"(71509.0, 90215.0)",0,\n \n \n \n Postdoctoral Fellow \n...,0,0,0,0,...,0,1,0,0,2,3.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209


In [45]:
import pandas as pd

# Assume df is your dataset with a column 'salary_range'
# Split the salary_range column into minimum and maximum columns
df[['minimum', 'maximum']] = pd.DataFrame(df['salary_string'].tolist(), index=df.index)

# Drop the original salary_range column
df.drop('salary_string', axis=1, inplace=True)


In [46]:
import pandas as pd

# Assume df is your dataset with columns 'salary_low' and 'salary_high'
# Calculate the median salary for each data point
df['salary_median'] = (df['minimum'] + df['maximum']) / 2

# Drop the 'salary_low' and 'salary_high' columns
df.drop(['minimum', 'maximum'], axis=1, inplace=True)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['teaser'], df['jobClassification'], test_size=0.2, random_state=42)

# Build the pipelines
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()
rf_pipeline = Pipeline([
    ('vect', count_vect),
    ('tfidf', tfidf_transformer),
    ('clf', RandomForestClassifier(random_state=42))
])
sgd_pipeline = Pipeline([
    ('vect', count_vect),
    ('tfidf', tfidf_transformer),
    ('clf', SGDClassifier(random_state=42))
])
svm_pipeline = Pipeline([
    ('vect', count_vect),
    ('tfidf', tfidf_transformer),
    ('clf', SVC(random_state=42))
])

# Fit the models
rf_pipeline.fit(X_train, y_train)
sgd_pipeline.fit(X_train, y_train)
svm_pipeline.fit(X_train, y_train)

# Evaluate the models
print('Random Forest Accuracy:', rf_pipeline.score(X_test, y_test))
print('SGD Accuracy:', sgd_pipeline.score(X_test, y_test))
print('SVM Accuracy:', svm_pipeline.score(X_test, y_test))


Random Forest Accuracy: 0.6463414634146342
SGD Accuracy: 0.6646341463414634
SVM Accuracy: 0.6402439024390244


In [50]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Define the pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearRegression())
])

# Train the model
text_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = text_clf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2_score = text_clf.score(X_test, y_test)

In [51]:
mse

4.067240415612213

In [52]:
rmse

2.016740046612903

In [53]:
r2_score

0.11330459979812046

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817 entries, 2 to 3893
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   jobClassification             817 non-null    float64
 1   teaser                        817 non-null    object 
 2   workType                      817 non-null    int64  
 3   isRightToWorkRequired         817 non-null    int64  
 4   desktopAdTemplate             817 non-null    object 
 5   Python                        817 non-null    int64  
 6   SQL                           817 non-null    int64  
 7   R                             817 non-null    int64  
 8   Tableau                       817 non-null    int64  
 9   SAS                           817 non-null    int64  
 10  Matlab                        817 non-null    int64  
 11  Hadoop                        817 non-null    int64  
 12  Spark                         817 non-null    int64  
 13  Java

In [55]:
df = df.drop(['teaser', 'desktopAdTemplate'],axis =1 )

In [62]:
df.head()

,jobClassification,workType,isRightToWorkRequired,Python,SQL,R,Tableau,SAS,Matlab,Hadoop,Spark,Java,Scala,recruiter,workType_encoded,jobClassification_encoded,teaser_cleaned,desktopAdTemplate_cleaned,teaser_word_count,desktopAdTemplate_word_count
2,0.0,2,0,0,1,1,0,0,0,0,0,0,0,1,2,0.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
3,1.0,2,0,0,1,1,0,1,0,0,0,0,0,1,2,1.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
7,1.0,2,1,1,1,1,0,1,0,0,0,0,0,1,2,1.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
10,2.0,2,0,0,0,0,0,0,0,0,0,1,0,0,2,2.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209
11,3.0,2,0,0,0,0,0,0,0,0,0,1,0,0,2,3.0,2 Fantastic organisation seeks experienced Ins...,2 \n \n \n \n Insights Analyst – Onl... 3 \n \...,4638,5209


In [57]:
from sklearn.model_selection import train_test_split

y = df.pop('salary_median')
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [61]:
from sklearn.preprocessing import MinMaxScaler

# Create an instance of the MinMaxScaler
scaler = MinMaxScaler()

# Scale the numeric columns in X
X_numeric = X.select_dtypes(include='number')
X_scaled = scaler.fit_transform(X_numeric)

# Convert the scaled numpy array back to a pandas dataframe
X_scaled_df = pd.DataFrame(X_scaled, columns=X_numeric.columns)

# Concatenate the scaled numeric columns with the text columns in X
X_processed = pd.concat([X_scaled_df, X.select_dtypes(include='object')], axis=1)

In [73]:
print(X.shape)
print(y.shape)

(817, 20)
(817,)


In [68]:
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.compose import ColumnTransformer
# from sklearn.metrics import r2_score

# text_features = ['teaser_cleaned', 'desktopAdTemplate_cleaned']
# # Define preprocessing pipelines for text and numeric features
# text_pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer())
# ])
# # Combine preprocessing pipelines for text and numeric features
# preprocessor = ColumnTransformer(transformers=[
#     ('text', text_pipeline, text_features)])

In [74]:
# # Define the linear regression model
# lr_model = Pipeline([
#     ('preprocessor', preprocessor),
#     ('lr', LinearRegression())
# ])

# # Fit the model to the training data
# lr_model.fit(X_train, y_train)

# # Predict on the test data
# y_pred = lr_model.predict(X_test)

# # Calculate the R-squared score
# r2_score = r2_score(y_test, y_pred)
# print('R-squared score:', r2_score)

In [78]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# # Split data into training and test sets
# y = df['salary_median']
# X = df.drop('salary_median', axis=1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create pipeline for Count Vectorizer, TF-IDF transformation, and linear regression
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('reg', LinearRegression())
])

# Fit pipeline on training data
pipeline.fit(X_train['desktopAdTemplate_cleaned'], y_train)

# Predict on test data
y_pred = pipeline.predict(X_test['desktopAdTemplate_cleaned'])


In [79]:
# Calculate the R-squared score
r2_score = r2_score(y_test, y_pred)
print('R-squared score:', r2_score)

R-squared score: -862015186.9423645


An R-squared score of -862015186.9423645 indicates a poor model fit. The R-squared score ranges from 0 to 1, with 1 indicating a perfect fit and 0 indicating that the model does not explain any of the variability in the target variable. A negative value means that the model performs worse than the baseline model that always predicts the mean of the target variable.

There might be several reasons why the model is performing so poorly, including:

Insufficient data: The model might be underfitting the data due to a lack of data.
Nonlinear relationship: The relationship between the features and the target variable might be nonlinear, which linear regression cannot capture.
Multicollinearity: There might be high multicollinearity between the features, which can cause numerical instability and produce unreliable coefficients.
Outliers: The presence of outliers in the data can greatly affect the performance of linear regression.
You may need to explore and preprocess the data further or consider using a different model that can better capture the relationship between the features and the target variable.